In [11]:
import numpy as np

In [12]:
annual_interest_rate = 0.03  # 3% interest rate
maturities = range(1, 10)  # Maturities from 1 to 9 years

# Computing the bond prices using the formula P(0, T) = 1 / (1 + r)^T
bond_prices = {T: 1 / ((1 + annual_interest_rate) ** T) for T in maturities}
bond_prices

{1: 0.970873786407767,
 2: 0.9425959091337544,
 3: 0.9151416593531596,
 4: 0.8884870479156888,
 5: 0.8626087843841639,
 6: 0.8374842566836542,
 7: 0.8130915113433536,
 8: 0.7894092343139355,
 9: 0.7664167323436267}

In [13]:
import numpy as np

# Model parameters
initial_forward_rate = 0.03  # Initial forward rate
sigma = 0.01  # Constant volatility
num_maturities = 9  # Number of forward rates (f0 to f9)
num_paths = 10000  # Number of simulation paths
time_step = 0.1  # Time step in years
total_time = 10  # Total simulation time in years

# Time points for simulation
num_steps = int(total_time / time_step)
time_points = np.linspace(0, total_time, num_steps + 1)

# Initializing matrix to store simulated forward rates for each path
forward_rates = np.zeros((num_paths, num_maturities, num_steps + 1))
forward_rates[:, :, 0] = initial_forward_rate

# Monte Carlo simulation
np.random.seed(0)  # For reproducibility
for path in range(num_paths):
    for step in range(num_steps):
        dt = time_points[step + 1] - time_points[step]
        dW = np.random.normal(0, np.sqrt(dt), num_maturities)  # Brownian motion increment
        forward_rates[path, :, step + 1] = forward_rates[path, :, step] + sigma * dW

# Averaging the forward rates across all paths
average_forward_rates = forward_rates.mean(axis=0)

# Displaying the average forward rates at the end of each year
average_forward_rates_at_year_end = average_forward_rates[:, ::int(1/time_step)]
average_forward_rates_at_year_end



array([[0.03      , 0.02986817, 0.0297884 , 0.02946869, 0.02945377,
        0.0294409 , 0.02953262, 0.02920268, 0.02943109, 0.02954459,
        0.02975883],
       [0.03      , 0.03012522, 0.0301151 , 0.03004778, 0.03001178,
        0.03004207, 0.03021245, 0.03032931, 0.03046166, 0.03033766,
        0.03039684],
       [0.03      , 0.03027252, 0.03006304, 0.02999198, 0.02999115,
        0.02996904, 0.02993888, 0.02997174, 0.029857  , 0.02992988,
        0.02996582],
       [0.03      , 0.02976336, 0.0298444 , 0.02984719, 0.0299841 ,
        0.03002855, 0.03007191, 0.02995904, 0.02981383, 0.02962971,
        0.02958304],
       [0.03      , 0.03025253, 0.03031811, 0.03037979, 0.03042168,
        0.03042989, 0.03042281, 0.03051057, 0.03065398, 0.0307732 ,
        0.03095046],
       [0.03      , 0.03011712, 0.03009441, 0.03002413, 0.0299437 ,
        0.03001127, 0.03005097, 0.0300236 , 0.02991907, 0.02976156,
        0.02975942],
       [0.03      , 0.02993203, 0.02980447, 0.02973093, 0.

In [14]:
# take average per column
average_forward_rates = average_forward_rates_at_year_end.mean(axis=0)
average_forward_rates

array([0.03      , 0.03002611, 0.03001695, 0.02996682, 0.03000169,
       0.03002379, 0.03005407, 0.03002514, 0.03004643, 0.03003944,
       0.03011859])

In [15]:
# Calculating the Monte Carlo values for discount bonds

# Initialize an array to store the Monte Carlo values of the bonds
mc_bond_values = np.zeros((num_maturities, num_steps + 1))

# Correcting the calculation for the Monte Carlo values of discount bonds

# Calculate the present value of each bond for each path and then average
for i in range(num_maturities):
    for path in range(num_paths):
        # Calculate the discount factor for each path at maturity T_i
        discount_factor = np.prod(1 + time_step * forward_rates[path, :i+1, i])
        mc_bond_values[i] += discount_factor

    # Average the discount factors over all paths
    mc_bond_values[i] /= num_paths

# Monte Carlo values of the bonds at t=0
mc_values_at_t0 = 1 / mc_bond_values[:, 0]
mc_values_at_t0





array([0.99700897, 0.99403134, 0.99104638, 0.988081  , 0.98512531,
       0.98216898, 0.97924711, 0.97632252, 0.97339843])

In [16]:
# Adjusted parameters for the Monte Carlo simulation using D_10(t) as the numeraire
num_maturities = 10  # Number of forward rates (f0 to f9, inclusive)
num_steps = num_maturities  # Total simulation time in years
time_step = 1  # Time step in years, Δ = 1
sigma = 0.01  # Constant volatility 1%

# Initializing matrix to store simulated forward rates for each path
forward_rates_mc = np.zeros((num_paths, num_maturities, num_steps + 1))
forward_rates_mc[:, :, 0] = initial_forward_rate

# Monte Carlo simulation under Q_10 measure
np.random.seed(0)  # For reproducibility
for path in range(num_paths):
    for step in range(num_steps):
        dW = np.random.normal(0, np.sqrt(time_step), num_maturities)  # Brownian motion increment
        forward_rates_mc[path, :, step + 1] = forward_rates_mc[path, :, step] + sigma * dW

# Calculating the Monte Carlo values for discount bonds under Q_10
mc_discount_bond_values = np.zeros((num_maturities, num_steps + 1))

# Calculating the present value of each bond for each path and then averaging
for i in range(num_maturities):
    for path in range(num_paths):
        # Calculate the discount factor for each path at maturity T_i
        discount_factor = np.prod(1 / (1 + time_step * forward_rates_mc[path, :i+1, i]))
        mc_discount_bond_values[i] += discount_factor

    # Average the discount factors over all paths
    mc_discount_bond_values[i] /= num_paths

# Extracting the Monte Carlo values of the bonds at t=0
mc_discount_values_at_t0 = mc_discount_bond_values[:, 0]
mc_discount_values_at_t0



array([0.97087379, 0.94283202, 0.91563443, 0.88968214, 0.86440704,
       0.83971669, 0.81588532, 0.79332587, 0.77152657, 0.74940902])